<a href="https://colab.research.google.com/github/AjayBora002/movie_recommendation/blob/main/Copy_of_movie_recommendation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1.Import Modules


In [ ]:
import pandas as pd
import numpy as np
import re
import nltk

In [ ]:
movies=pd.read_csv("movies.csv")

In [ ]:


def clean_title(title):
    return re.sub(r'[^a-zA-Z0-9 ]', '', title.lower()).strip()


In [ ]:
movies["clean_title"]=movies["title"].apply(clean_title)# this will access a new clean title column


2. FINDING SIMILARITIES BETWEEN THE WORDS AND WITH THE HELP OF IT GIVING THE SIMILAR TYPE OF MOVIE TO THE USER

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer# converting text data into numerical data and finding similarities , this will do it
vectorizer = TfidfVectorizer(ngram_range=(1,2))# this ngram thing will help in finding instead of single single word it will match two consecutive words like toy story together
tfidf = vectorizer.fit_transform(movies["clean_title"])

3.finding similarity and creating search function

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

def search(title):
    title = clean_title(title)  # clean the input title
    query_vec = vectorizer.transform([title])  # vectorize it
    similarity = cosine_similarity(query_vec, tfidf).flatten()
    indices = np.argpartition(similarity, -5)[-5:]
    results = movies.iloc[indices].iloc[::-1]
    return results


4.CREATING AN INTERACTIVE BOX

In [ ]:
import ipywidgets as widgets
from IPython.display import display  # function used to show diiff things as output on notebook


movie_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)


# this whole thing will create a box to enter movie title , but wont show anything as output
  #NOW MAKING AN OUTPUT WIDGET
movie_list = widgets.Output()

def on_type(data):     # this fun will be called whenever we type something in the box
  with movie_list:
    movie_list.clear_output()

    title=data["new"]
    if len(title)>5:
      display(search(title))

movie_input.observe(on_type, names='value')  # there are diff events of the widgets whenever we input a movie name it is going to call on type and gives value event
display(movie_input, movie_list)




In [ ]:
ratings=pd.read_csv("ratings.csv")

In [ ]:
ratings.dtypes

5.FINDING SIMILAR USER RECOMMENDATION

In [ ]:
movie_id=1

In [ ]:
similar_users = ratings[(ratings["movieId"] == movie_id)# this is for finding anyone who watched same movie
& (ratings["rating"] > 4)]["userId"].unique()# this is for anyone who watched the movie and gave the rating 5

similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) & (ratings["rating"] > 4)]["movieId"]
# this will show the movies that are rated by similar users and find their used ids

In [ ]:
similar_user_recs

In [ ]:
similar_user_recs=similar_user_recs.value_counts()/len(similar_users) # this counts the no of 5 rates is given to a movie and then div by len shows recommending percent like top one has 100%

# we are doing this to find per of similar people who watched the movie
similar_user_recs = similar_user_recs[similar_user_recs > .10]  # shows movies having per greater than 10 %

In [ ]:
similar_user_recs

6.now problem is that most of the people like movies like toy story and avengers , what we need is how much movies are liked by users are we also like this will give more accuracy on recommendation

In [ ]:
all_users=ratings[(ratings)["movieId"].isin(similar_user_recs.index) & (ratings["rating"]>4)]

In [ ]:
all_users_recs=all_users["movieId"].value_counts()/len(all_users["userId"].unique())# this will give us percentage of all the users who liked the movies

In [ ]:
rec_percentages=pd.concat([similar_user_recs,all_users_recs],axis=1)# this concatenates the data of how much similar people to us liked and how much avg person liked it
rec_percentages.columns=["similar","all"]

In [ ]:
rec_percentages

7.NOW WE WANT MOVIES THAT HAS A BIG DIFF BET THESE TWO COLUMN

In [ ]:
rec_percentages["score"]=rec_percentages["similar"]/rec_percentages["all"]
rec_percentages=rec_percentages.sort_values("score",ascending=False)   # using pandas sort method to make an order of the score the higher the score more good will be the recommendation
rec_percentages

8.NOW TAKING TOP 10 RATINGS AND MERGE IT WITH MOVIES DATA

In [ ]:
rec_percentages.head(10).merge(movies,left_index=True,right_on="movieId")

9.PUTTING ALL OF THIS INTO A FUNCTION

In [ ]:
def find_similar_movies(movie_id):

    similar_users = ratings[(ratings["movieId"] == movie_id)
    & (ratings["rating"] > 4)]["userId"].unique()
    similar_user_recs = ratings[(ratings["userId"].isin(similar_users)) # this is finding recommendation similar to us
    & (ratings["rating"] > 4)]["movieId"]

    similar_user_recs = similar_user_recs.value_counts() / len(similar_users)# adjusting the per of recommendation
    similar_user_recs = similar_user_recs[similar_user_recs > .10]

    all_users = ratings[(ratings["movieId"].isin(similar_user_recs.index)) & (ratings["rating"] > 4)]# this is finding common recommendation among all of the users
    all_user_recs = all_users["movieId"].value_counts() / len(all_users["userId"].unique())

    rec_percentages = pd.concat([similar_user_recs, all_user_recs], axis=1)# concatinating
    rec_percentages.columns = ["similar", "all"]

    rec_percentages["score"] = rec_percentages["similar"] / rec_percentages["all"]# generating score

    rec_percentages=rec_percentages.sort_values("score", ascending=False)
    return rec_percentages.head(10).merge(movies, left_index=True, right_on="movieId")[["score", "title", "genres"]]  #sorting and returning our merge data

10.CREATING AN INTERACTIVE RECOMMENDATION WIDGET

In [ ]:
movie_name_input = widgets.Text(    # creating a widget
    value='Toy Story',
    description = "Movie Title :",  # movie titled will be entered here
    disabled=False   # it will be enabled
)

recommendation_list=widgets.Output()

def on_type(data):
  with recommendation_list:
    recommendation_list.clear_output()# removes old output
    title=data["new"]
    if len(title)>5:
      results=search(title)
      movie_id=results.iloc[0]["movieId"]
      display(find_similar_movies(movie_id))

movie_name_input.observe(on_type,names="value")
display(movie_name_input, recommendation_list)